- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- (опциционально)сравнить с точностью полносвязной сети для этой выборки

In [22]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow import keras
from tensorflow.keras.models import Model 

# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

In [4]:
# инициализация  модели
input1= layers.Input(shape=(28,28,1))
# первый сверточный слой
x1 = layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh',  padding="same")(input1)

# второй пуллинговый слой
x2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)

# третий сверточный слой
x3 = layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')(x2)

# четвертый пуллинговый слой
x4 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)

# пятый  слой
x5 = layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')(x4)

# сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
x6 = layers.Flatten()(x5)

# шестой полносвязный слой
x7 = layers.Dense(84, activation='tanh')(x6)

# выходной слой с функцией активации softmax
out_x = layers.Dense(10, activation='softmax')(x7)


# Соберем полную модель сети от входа к выходу 
model1 = Model(inputs = input1, outputs = out_x) 
# сделаем несколько промежуточных выходов (через них посмотрим , что происходит в сети) 
model3 = Model(inputs = input1, outputs = x3)
model5 = Model(inputs = input1, outputs = x5)
# компилияция модели
model1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

# Обучаем модель
hist = model1.fit(x=x_train,y=y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model1.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/3
469/469 [==============================] - 37s 79ms/step - loss: 0.9207 - accuracy: 0.6818 - val_loss: 0.6980 - val_accuracy: 0.7328
Epoch 2/3
469/469 [==============================] - 40s 84ms/step - loss: 0.6144 - accuracy: 0.7714 - val_loss: 0.6024 - val_accuracy: 0.7788
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - loss: 0.5588 - accuracy: 0.7940
Test loss 0.5588, accuracy 79.40%


### Изменение количества ядер

In [5]:
def test_kernel_size(kernel_size):
    # инициализация  модели
    input1= layers.Input(shape=(28,28,1))
    # первый сверточный слой
    x1 = layers.Conv2D(6, kernel_size=kernel_size, strides=(1, 1), activation='tanh',  padding="same")(input1)

    # второй пуллинговый слой
    x2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)

    # третий сверточный слой
    x3 = layers.Conv2D(16, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x2)

    # четвертый пуллинговый слой
    x4 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)

    # пятый  слой
    x5 = layers.Conv2D(120, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x4)

    # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
    x6 = layers.Flatten()(x5)

    # шестой полносвязный слой
    x7 = layers.Dense(84, activation='tanh')(x6)

    # выходной слой с функцией активации softmax
    out_x = layers.Dense(10, activation='softmax')(x7)


    # Соберем полную модель сети от входа к выходу 
    model1 = Model(inputs = input1, outputs = out_x) 
    # сделаем несколько промежуточных выходов (через них посмотрим , что происходит в сети) 
    model3 = Model(inputs = input1, outputs = x3)
    model5 = Model(inputs = input1, outputs = x5)
    # компилияция модели
    model1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

    # Обучаем модель
    hist = model1.fit(x=x_train,y=y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test), verbose=1)

    test_score = model1.evaluate(x_test, y_test)
    print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

In [6]:
test_kernel_size((8, 8))

Epoch 1/3
469/469 [==============================] - 50s 105ms/step - loss: 0.8882 - accuracy: 0.6926 - val_loss: 0.6682 - val_accuracy: 0.7464
Epoch 2/3
469/469 [==============================] - 49s 104ms/step - loss: 0.5847 - accuracy: 0.7830 - val_loss: 0.5787 - val_accuracy: 0.7891
Epoch 3/3
313/313 [==============================] - 2s 7ms/step - loss: 0.5191 - accuracy: 0.8115
Test loss 0.5191, accuracy 81.15%


Вывод: При увеличении ширины метрика улучшилась.

### Изменение количества слоёв

In [20]:
kernel_size = (5, 5)

In [23]:
# инициализация  модели
input1= layers.Input(shape=(28,28,1))
# первый сверточный слой
x1 = layers.Conv2D(6, kernel_size=kernel_size, strides=(1, 1), activation='tanh',  padding="same")(input1)

# второй пуллинговый слой
x2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x1)

# третий сверточный слой
x3 = layers.Conv2D(16, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x2)

# дополнительный сверточный слой
x4 = layers.Conv2D(16, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x3)

# дополнительный сверточный слой
x5 = layers.Conv2D(16, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x4)

# дополнительный сверточный слой
x6 = layers.Conv2D(16, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x5)

# четвертый пуллинговый слой
x7 = layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(x6)

# пятый  слой
x8 = layers.Conv2D(120, kernel_size=kernel_size, strides=(1, 1), activation='tanh', padding='valid')(x7)

# сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
x9 = layers.Flatten()(x8)

# шестой полносвязный слой
x10 = layers.Dense(84, activation='tanh')(x9)

# выходной слой с функцией активации softmax
out_x = layers.Dense(10, activation='softmax')(x10)


# Соберем полную модель сети от входа к выходу 
model1 = Model(inputs = input1, outputs = out_x) 
# сделаем несколько промежуточных выходов (через них посмотрим , что происходит в сети) 
model3 = Model(inputs = input1, outputs = x3)
model5 = Model(inputs = input1, outputs = x5)
# компилияция модели
model1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

# Обучаем модель
hist = model1.fit(x=x_train,y=y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model1.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/3
469/469 [==============================] - 64s 136ms/step - loss: 0.9398 - accuracy: 0.6704 - val_loss: 0.7110 - val_accuracy: 0.7330
Epoch 2/3
469/469 [==============================] - 77s 164ms/step - loss: 0.6361 - accuracy: 0.7599 - val_loss: 0.6209 - val_accuracy: 0.7673
Epoch 3/3
313/313 [==============================] - 3s 11ms/step - loss: 0.5715 - accuracy: 0.7851
Test loss 0.5715, accuracy 78.51%


Вывод: предложенный вариант измененной архитектуры слоя, с увеличенным количеством свёрточных слоев, показал падение метрики.

### Сравнение с полносвязной сетью

In [33]:
from keras.models import Sequential
from keras.layers import Dense

(train_imagesi,train_labels),(test_imagesi,testn_labels)=fashion_mnist.load_data()

# Normalize the images.
train_imagesi = (train_imagesi / 127) - 1
test_imagesi = (test_imagesi / 127) - 1

# Flatten the images.
x_train = train_imagesi.reshape((-1, 784))
x_test = test_imagesi.reshape((-1, 784))

y_t = to_categorical(train_labels)
y_tt = to_categorical(testn_labels)

In [34]:
model1 = Sequential(name='my_model')
model1.add(Dense(256, activation='relu', input_shape=(784,)))
model1.add(Dense(10, activation='softmax'))

model1.compile(
  optimizer='adam',
  loss=keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'],
)

# Обучаем модель
hist = model1.fit(x=x_train,y=y_t, epochs=3, batch_size=128, validation_data=(x_test, y_tt), verbose=1)

test_score = model1.evaluate(x_test, y_tt)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/3
469/469 [==============================] - 1s 2ms/step - loss: 0.4945 - accuracy: 0.8228 - val_loss: 0.4436 - val_accuracy: 0.8350
Epoch 2/3
469/469 [==============================] - 1s 2ms/step - loss: 0.3712 - accuracy: 0.8658 - val_loss: 0.3971 - val_accuracy: 0.8570
Epoch 3/3
313/313 [==============================] - 0s 921us/step - loss: 0.3841 - accuracy: 0.8591
Test loss 0.3841, accuracy 85.91%


Вывод: предложенные примеры свёрточных сетей показывают меньшие accuracy.